In [1]:
!pip install -U langchain

In [2]:
!pip install langchain_experimental

In [3]:
!pip install faiss-cpu

In [4]:
import os

os.environ["SERPAPI_API_KEY"] = "e65622355785aba531fe0f3733c6c429e3ec43457c916a0c3006e6f81d433369"

In [5]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


In [6]:
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [7]:
from langchain.embeddings import OpenAIEmbeddings

In [8]:
# OpenAI Embedding 模型
embeddings_model = OpenAIEmbeddings()

In [10]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# OpenAI Embedding 向量维数
embedding_size = 1536
# 使用 Faiss 的 IndexFlatL2 索引
index = faiss.IndexFlatL2(embedding_size)
# 实例化 Faiss 向量数据库
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## 实例化自主智能体 Auto-GPT

In [11]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

In [12]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(), # 实例化 Faiss 的 VectorStoreRetriever
)

In [13]:
# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True

In [14]:
agent.run(["2023年大运会举办地在哪？"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年大运会举办地在哪？


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"tit

'I have finished all my objectives.'

In [15]:
agent.run(["2023年大运会举办地在哪？", "2023年大运会中国队奖牌数多少？"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年大运会举办地在哪？
2. 2023年大运会中国队奖牌数多少？


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema

'I have finished all my objectives.'

In [19]:
gpt_4_agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model_name="gpt-4", temperature=0),
    memory=vectorstore.as_retriever(), # 实例化 Faiss 的 VectorStoreRetriever
)

In [20]:
gpt_4_agent.run(["2023年大运会举办地在哪？", "2023年大运会中国队奖牌数多少？"])

{
    "thoughts": {
        "text": "I need to determine the location of the 2023 World University Games. I can use the 'search' command to find this information.",
        "reasoning": "I don't have this information in my short-term memory, so I need to search for it.",
        "plan": "- Use the 'search' command to find the location of the 2023 World University Games.",
        "criticism": "None",
        "speak": "I will search for the location of the 2023 World University Games."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "2023 World University Games location"
        }
    }
}
{
    "thoughts": {
        "text": "The search command returned the information that the 2023 World University Games will be held in North America. I should save this information for future reference.",
        "reasoning": "I need to save the location of the 2023 World University Games for future use.",
        "plan": "- Use the 'write_file' command to s

'I have completed all my tasks.'

In [22]:
gpt_4_agent.chain.verbose = True

In [23]:
gpt_4_agent.run(["Auto-GPT 是什么？把结果写到autogpt.txt文件中"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Auto-GPT 是什么？把结果写到autogpt.txt文件中


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {

'I have completed all my tasks.'